<a href="https://colab.research.google.com/github/women-in-ai-ireland/September-2022-CeADAR-Group-002/blob/main/DataPreparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%reset -f

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

dicCounties={'Carlow':['Carlow'],'Cavan':['Cavan'],'Clare':['Clare'],'Cork':['Cork'],'Donegal':['Donegal'],'Dublin':['Dublin','Fingal','Dún Laoghaire'],
             'Galway':['Galway'],'Kerry':['Kerry'],'Kildare':['Kildare'],'Kilkenny':['Kilkenny'],'Laois':['Laois'],'Leitrim':['Leitrim'],
             'Limerick':['Limerick'],'Longford':['Longford'],'Louth':['Louth'],'Mayo':['Mayo'],'Meath':['Meath'],'Monaghan':['Monaghan'],
             'Offaly':['Offaly'],'Roscommon':['Roscommon'],'Sligo':['Sligo'],'Tipperary':['Tipperary'],'Waterford':['Waterford'],
             'Westmeath':['Westmeath'],'Wexford':['Wexford'],'Wicklow':['Wicklow']};

# lstQuarters=['2011Q1','2011Q2','2011Q3','2011Q4','2012Q1','2012Q2','2012Q3','2012Q4','2013Q1','2013Q2','2013Q3','2013Q4','2014Q1','2014Q2','2014Q3','2014Q4',
#              '2015Q1','2015Q2','2015Q3','2015Q4','2016Q1','2016Q2','2016Q3','2016Q4','2017Q1','2017Q2','2017Q3','2017Q4','2018Q1','2018Q2','2018Q3','2018Q4',
#              '2019Q1','2019Q2','2019Q3','2019Q4','2020Q1','2020Q2','2020Q3','2020Q4','2021Q1','2021Q2','2021Q3','2021Q4','2022Q1','2022Q2'];

Quarters={'Q1':['January','February','March'],
          'Q2':['April','May','June'],
          'Q3':['July','August','September'],
          'Q4':['October','November','December']
          }

In [3]:
dfNDQ06=pd.read_csv("https://raw.githubusercontent.com/women-in-ai-ireland/September-2022-CeADAR-Group-002/main/dataset/NDQ06.20221013.csv")
dfNDQ06=dfNDQ06[dfNDQ06["Type of House"]=="All house types"]

dfNDQ06['County']=np.nan

for county in dicCounties:
  filter=dfNDQ06['Local Authority'].str.contains('|'.join(dicCounties[county]),case=False,regex=True,na=False)
  dfNDQ06.loc[filter, 'County'] = county

dfNDQ06.drop(columns=['STATISTIC','Type of House','UNIT','Local Authority'],inplace=True)
dfNDQ06.rename(columns={'VALUE':'New_Dwelling_Completion_Number'},inplace=True)
dfNDQ06=dfNDQ06.groupby(['County','Quarter']).sum()

dfGrand=dfNDQ06.copy()

In [4]:
dfHPM02=pd.read_csv("https://raw.githubusercontent.com/women-in-ai-ireland/September-2022-CeADAR-Group-002/main/dataset/HPM02.20221018.csv")
dfHPM02=dfHPM02[dfHPM02["Dwelling Status"]=="All Dwelling Statuses"]
dfHPM02=dfHPM02[dfHPM02["Stamp Duty Event"]=="Executions"]
dfHPM02.drop(columns=['Dwelling Status','Type of Buyer','Type of Sale','Stamp Duty Event'],inplace=True)
dfHPM02

def setQuarter(month):
  split=month.split()
  quarter = [i for i in Quarters if split[1] in Quarters[i]]
  return split[0]+quarter[0]


dfHPM02["Quarter"]=dfHPM02['Month'].apply(setQuarter)

dfHPM02tmp=dfHPM02[dfHPM02['Statistic']=='Volume of Sales']
dfGrand['Volume_of_Sales_Number']=dfHPM02tmp.groupby(['County','Quarter']).sum()

dfHPM02tmp=dfHPM02[dfHPM02['Statistic']=='Value of Sales']
dfGrand['Value_of_Sales_MilEuro']=dfHPM02tmp.groupby(['County','Quarter']).sum()

dfHPM02tmp=dfHPM02[dfHPM02['Statistic']=='Median Price']
dfGrand['Median_Price_Euro']=dfHPM02tmp.groupby(['County','Quarter']).mean()

dfHPM02tmp=dfHPM02[dfHPM02['Statistic']=='Mean Sale Price']
dfGrand['Mean_Price_Euro']=dfHPM02tmp.groupby(['County','Quarter']).mean()

In [5]:
dfE1071=pd.read_csv("https://raw.githubusercontent.com/women-in-ai-ireland/September-2022-CeADAR-Group-002/main/dataset/E1071.20221013.csv")
dfFP012=pd.read_csv("https://raw.githubusercontent.com/women-in-ai-ireland/September-2022-CeADAR-Group-002/main/dataset/FP012.20221013.csv")

dfE1071['Quarter']=dfE1071['CensusYear'].apply(lambda x:str(x)+'Q2')
dfE1071['County']=np.nan
for county in dicCounties:
  filter=dfE1071['County and City'].str.contains('|'.join(dicCounties[county]),case=False,regex=True,na=False)
  dfE1071.loc[filter, 'County'] = county

dfE1071=dfE1071.set_index(['County','Quarter'])

dfFP012['Quarter']=dfFP012['CensusYear'].apply(lambda x:str(x)+'Q2')
dfFP012['County']=np.nan
for county in dicCounties:
  filter=dfFP012['Admin Counties'].str.contains('|'.join(dicCounties[county]),case=False,regex=True,na=False)
  dfFP012.loc[filter, 'County'] = county

dfFP012=dfFP012.set_index(['County','Quarter'])

In [6]:
dfE1071tmp=dfE1071[dfE1071['Statistic']=='Total housing stock']
i=dfE1071tmp.groupby(['County','Quarter']).sum() 
dfGrand['Housing_Stock_Number']=i['VALUE']

dfE1071tmp=dfE1071[dfE1071['Statistic']=='Vacant']
i=dfE1071tmp.groupby(['County','Quarter']).sum() 
dfGrand['Vacant_House_Number']=i['VALUE']

dfFP012tmp=dfFP012[dfFP012['Statistic']=='Housing Stock']
i=dfFP012tmp.groupby(['County','Quarter']).sum()
dfGrand.loc[(slice(None),'2022Q2'),'Housing_Stock_Number']=i['VALUE']

dfFP012tmp=dfFP012[dfFP012['Statistic']=='Vacant Dwellings']
i=dfFP012tmp.groupby(['County','Quarter']).sum()
dfFP012tmp=dfFP012[dfFP012['Statistic']=='Unoccupied Holiday Homes']
j=dfFP012tmp.groupby(['County','Quarter']).sum()

dfGrand.loc[(slice(None),'2022Q2'),'Vacant_House_Number']=i['VALUE']+j['VALUE']


for county in dfGrand.index.get_level_values(0).unique():
  dfGrand.loc[county]['Housing_Stock_Number']=dfGrand.loc[county,'Housing_Stock_Number'].interpolate(method='linear',limit_direction='both')
  dfGrand.loc[county]['Vacant_House_Number']=dfGrand.loc[county,'Vacant_House_Number'].interpolate(method='linear',limit_direction='both')


dfGrand.to_csv('HousingData.csv')
dfGrand

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


New_Dwelling_Completion_Number  Volume_of_Sales_Number  \
County  Quarter                                                           
Carlow  2011Q1                               17                    50.0   
        2011Q2                               24                    59.0   
        2011Q3                               24                    62.0   
        2011Q4                               38                    74.0   
        2012Q1                               17                    48.0   
...                                         ...                     ...   
Wicklow 2021Q2                              149                   526.0   
        2021Q3                              229                   639.0   
        2021Q4                              239                   697.0   
        2022Q1                              268                   420.0   
        2022Q2                              332                   591.0   

                 Value_of_Sales_MilEuro  Median_Price_Euro  Mean_Price_Euro  \
County  Quarter                                                               
Carlow  2011Q1                      6.8      153205.000000    136542.666667   
        2011Q2                      6.6      110000.000000    111140.666667   
        2011Q3                      5.9       77833.333333     96286.000000   
        2011Q4                      6.1       71848.333333     81263.000000   
        2012Q1                      3.9       77666.666667     80014.000000   
...                                 ...                ...              ...   
Wicklow 2021Q2                    186.2      323500.000000    348426.000000   
        2021Q3                    250.5      342500.000000    392000.333333   
        2021Q4                    281.5      375116.333333    401860.333333   
        2022Q1                    151.1      337367.666667    361707.000000   
        2022Q2                    255.4      403888.666667    431266.333333   

                 Housing_Stock_Number  Vacant_House_Number  
County  Quarter                                             
Carlow  2011Q1              23165.000             3202.000  
        2011Q2              23165.000             3202.000  
        2011Q3              23170.450             3150.000  
        2011Q4              23175.900             3098.000  
        2012Q1              23181.350             3046.000  
...                               ...                  ...  
Wicklow 2021Q2              59258.500             4446.500  
        2021Q3              59472.125             4446.125  
        2021Q4              59685.750             4445.750  
        2022Q1              59899.375             4445.375  
        2022Q2              60113.000             4445.000  

[1196 rows x 7 columns]